In [1]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
from datetime import date, datetime
from pytz import timezone
import plotly.express as px
import numpy as np

In [2]:
df=pd.read_csv('stock.csv')

In [127]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [128]:
df = pd.read_sql_query("SELECT * FROM tsla_stock_prices", con)

In [3]:
df.head()

,Unnamed: 0,open,high,low,last,close,volume,date,symbol,exchange
0,0,752.920,752.920,752.920,NaN,NaN,NaN,2021-09-08T00:00:00+0000,TSLA,IEXG
1,1,753.065,753.495,752.950,NaN,NaN,NaN,2021-09-07T23:30:00+0000,TSLA,IEXG
2,2,754.850,755.170,754.610,NaN,NaN,NaN,2021-09-07T23:00:00+0000,TSLA,IEXG
3,3,752.890,753.030,752.470,NaN,NaN,NaN,2021-09-07T22:30:00+0000,TSLA,IEXG
4,4,754.445,754.780,754.445,NaN,NaN,NaN,2021-09-07T22:00:00+0000,TSLA,IEXG


In [4]:
df_2_columns = df[['last', 'date']]

In [5]:
df_2_columns.head()

,last,date
0,NaN,2021-09-08T00:00:00+0000
1,NaN,2021-09-07T23:30:00+0000
2,NaN,2021-09-07T23:00:00+0000
3,NaN,2021-09-07T22:30:00+0000
4,NaN,2021-09-07T22:00:00+0000


In [6]:
for index, row in df_2_columns['date'].iteritems():
    df_2_columns.loc[index, 'date'] = row.replace('T', ' ').replace('+', '')

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [7]:
df_2_columns

,last,date
0,NaN,2021-09-08 00:00:000000
1,NaN,2021-09-07 23:30:000000
2,NaN,2021-09-07 23:00:000000
3,NaN,2021-09-07 22:30:000000
4,NaN,2021-09-07 22:00:000000
...,...,...
95,737.570,2021-08-31 18:30:000000
96,737.200,2021-08-31 18:00:000000
97,736.670,2021-08-31 17:30:000000
98,738.440,2021-08-31 17:00:000000


In [8]:
eastern = timezone('US/Eastern')
utc = timezone('UTC')

# "%d/%m/%Y %H:%M:%S"
for index, row in df_2_columns['date'].iteritems():
    
    created_at = datetime.strptime(row[:-5], '%Y-%m-%d %H:%M:%S')
    utc_created_at = utc.localize(created_at)
    #print(utc_created_at)
    row = utc_created_at.astimezone(eastern)
    df_2_columns.loc[index, 'date_est'] = str(row)

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [9]:
df_2_columns
df_2_columns.drop(columns = 'date')

,last,date_est
0,NaN,2021-09-07 20:00:00-04:00
1,NaN,2021-09-07 19:30:00-04:00
2,NaN,2021-09-07 19:00:00-04:00
3,NaN,2021-09-07 18:30:00-04:00
4,NaN,2021-09-07 18:00:00-04:00
...,...,...
95,737.570,2021-08-31 14:30:00-04:00
96,737.200,2021-08-31 14:00:00-04:00
97,736.670,2021-08-31 13:30:00-04:00
98,738.440,2021-08-31 13:00:00-04:00


In [10]:
for index, row in df_2_columns['date_est'].iteritems():
    #print(row[:10])
    # time is currently utc
    df_2_columns.loc[index, 'time'] = row[11:16]
    df_2_columns.loc[index, 'new_date'] = row[:10]
df_2_columns

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,last,date,date_est,time,new_date
0,NaN,2021-09-08 00:00:000000,2021-09-07 20:00:00-04:00,20:00,2021-09-07
1,NaN,2021-09-07 23:30:000000,2021-09-07 19:30:00-04:00,19:30,2021-09-07
2,NaN,2021-09-07 23:00:000000,2021-09-07 19:00:00-04:00,19:00,2021-09-07
3,NaN,2021-09-07 22:30:000000,2021-09-07 18:30:00-04:00,18:30,2021-09-07
4,NaN,2021-09-07 22:00:000000,2021-09-07 18:00:00-04:00,18:00,2021-09-07
...,...,...,...,...,...
95,737.570,2021-08-31 18:30:000000,2021-08-31 14:30:00-04:00,14:30,2021-08-31
96,737.200,2021-08-31 18:00:000000,2021-08-31 14:00:00-04:00,14:00,2021-08-31
97,736.670,2021-08-31 17:30:000000,2021-08-31 13:30:00-04:00,13:30,2021-08-31
98,738.440,2021-08-31 17:00:000000,2021-08-31 13:00:00-04:00,13:00,2021-08-31


In [13]:
df_2_columns = df_2_columns.drop(columns = 'date')

df_2_columns = df_2_columns.drop(columns = 'date_est')

In [14]:
df_3 = df_2_columns

In [15]:
df_3.head()

,last,time,new_date
0,NaN,20:00,2021-09-07
1,NaN,19:30,2021-09-07
2,NaN,19:00,2021-09-07
3,NaN,18:30,2021-09-07
4,NaN,18:00,2021-09-07


In [16]:
df_4 = df_3[df_3['new_date'].str.match('2021-09-03')]
df_4.head()

df_4 = df_4.sort_values(by = ['time'])

In [17]:
df_4.head()

,last,time,new_date
43,734.78,09:00,2021-09-03
42,731.61,09:30,2021-09-03
41,731.93,10:00,2021-09-03
40,730.00,10:30,2021-09-03
39,730.18,11:00,2021-09-03


In [26]:
df_5 = pd.DataFrame()
df_5

""


In [27]:
for index, row in df_4.iterrows():
    #print(row.to_frame())
    if row['time'] != '16:30':
        df_5 = df_5.append(row)
    else:
        break
df_5 = df_5.drop_duplicates().reset_index(drop = True)
df_5 = df_5.fillna(df_5.mean())
df_5

,last,new_date,time
0,734.780000,2021-09-03,09:00
1,731.610000,2021-09-03,09:30
2,731.930000,2021-09-03,10:00
3,730.000000,2021-09-03,10:30
4,730.180000,2021-09-03,11:00
5,727.650000,2021-09-03,11:30
6,728.110000,2021-09-03,12:30
7,730.090000,2021-09-03,13:00
8,728.640000,2021-09-03,13:30
9,726.440000,2021-09-03,14:00


In [28]:
fig = px.line(df_5, x = 'time', y = 'last', title = 'Stock Price on 2021-09-03', 
             labels = {
                 'time': 'Time of Day (half hour)',
                 'last': 'Stock Price ($)',
             })
fig.show()

In [30]:
df_5.to_csv('stock_graph.csv')